In [5]:
# Import packages
import pandas as pd
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterio
from statsmodels.tsa.seasonal import STL
import numpy as np
from sklearn.linear_model import LinearRegression
import datetime as dt
from matplotlib.colors import TwoSlopeNorm
import time
from rasterstats import zonal_stats
from osgeo import gdal
import re

In [6]:
# This function rescales the pixel value such that we derive the physical values
def rescale(j, nodata_value_new, Scaling, Offset, file_path):
    tiff_open = gdal.Open(file_path) # open the GeoTIFF we want to rescale
    band = tiff_open.GetRasterBand(1) # Select the band
    tiff_array = tiff_open.ReadAsArray() # Assign raster values to a numpy nd array

    nodata_value_old = tiff_open.GetRasterBand(1).GetNoDataValue() # value which has been assigned for the nodata
    #nodata_value_new = -9999 # new data value

    tiff_array_new = np.where(tiff_array != nodata_value_old, tiff_array * Scaling + Offset, nodata_value_new) # Rescale pixel values and change nodatavalue to -9999

    col = tiff_open.RasterXSize # number of columns
    rows = tiff_open.RasterYSize # number of rows
    driver = tiff_open.GetDriver()

    # Create a new raster to save the average-values
    new_raster = driver.Create(j, col, rows, 1, gdal.GDT_Float32)

    # Copy the properties of the original raster
    new_raster.SetGeoTransform(tiff_open.GetGeoTransform())
    new_raster.SetProjection(tiff_open.GetProjection())

    # Add the average values to newly created raster
    new_raster.GetRasterBand(1).WriteArray(tiff_array_new)
    new_raster.GetRasterBand(1).SetNoDataValue(nodata_value_new)

    # close the raster
    new_raster = None
    del new_raster

Before running the code, we should set Offset and Scaling equal to as provided on the website. Furthermore, create the folders to which we will save the rescaling GeoTIFFs.

In [7]:
# For SSM, we do not need to rescale. Hence, we only change the no data value
offset = 0 
scaling = 1
ndv = -9999

# For each of the SSM images, we rescale the Geotiff
path = 'C:/Users/mieke/Documents/Msc Thesis/Datasets/GEE/SSM/Telangana/aoi/'
file_names = os.listdir(path) # contains all filenames
os.chdir('C:/Users/mieke/Documents/Msc Thesis/Datasets/GEE/SSM/Telangana/aoi_rescaled/') # Set path
for i in file_names:
    tiff_path = path + i # the selected GeoTIFF path
    rescale(i, ndv, scaling, offset, tiff_path)

Below, we find the exact same approach used for Karnataka.

In [8]:
# For SSM, we do not need to rescale. Hence, we only change the no data value
offset = 0 
scaling = 1
ndv = -9999

# For each of the SSM images, we rescale the Geotiff
path = 'C:/Users/mieke/Documents/Msc Thesis/Datasets/GEE/SSM/Karnataka/aoi/'
file_names = os.listdir(path) # contains all filenames
os.chdir('C:/Users/mieke/Documents/Msc Thesis/Datasets/GEE/SSM/Karnataka/aoi_rescaled/') # Set path
for i in file_names:
    tiff_path = path + i # the selected GeoTIFF path
    rescale(i, ndv, scaling, offset, tiff_path)

The rescaling is finished. Below is just a small check.

In [10]:
path_r = 'C:/Users/mieke/Documents/Msc Thesis/Datasets/GEE/SSM/Karnataka/aoi_rescaled/' + i # the path of the selected GeoTIFF
SSM_r = rasterio.open(path_r, mode = 'r') # open the selected GeoTIFF
SSM_r_array = SSM_r.read(1) # Assign raster values to a numpy nd array
row_r_34 = SSM_r_array[34]
print(row_r_34[23])
type(row_r_34[23])
print(SSM_r.nodata)
SSM_r.meta
SSM_r.transform # Note that the first element shows the resolution. To translate the resolution to metres, we use element * 10^5

6.770902
-9999.0


Affine(0.11481983169296148, 0.0, 73.9439716102672,
       0.0, -0.11481983169296148, 18.485992902566796)